In [1]:
import keras
import tensorflow as tf
import os
import numpy as np

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print('Length of the text: {} characters'.format(len(text)))

Length of the text: 1115394 characters


In [10]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [4]:
vocab = sorted(set(text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

In [5]:
text_as_int = text_to_int(text)

In [13]:
print('{} ---- characters mapped to int ---- > {}'.format(text[:13], text_as_int[:13]))

First Citizen ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [6]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

In [15]:
print(int_to_text(text_as_int[:13]))

First Citizen


In [7]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [8]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [9]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [19]:
for x, y in dataset.take(2):
  print('\n\n Example \n')
  print("INPUT")
  print(int_to_text(x))
  print("TARGET")
  print(int_to_text(y))



 Example 

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You
TARGET
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


 Example 

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 
TARGET
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [10]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab) # number of unique chraracters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# buffer size to shuffle the data
# ( tf data is designed to work with possibly infinte sequences, so it doesn't attempt to shuffle the entire sequence in memory. Instead it maintains a buffer in memory and shuffles the buffer)

BUFFER_SIZE = 10000

data = dataset.shuffle(buffer_size=BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [11]:
def build_model(vocab_size, embedding_dim, rnn_unit, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_unit, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [12]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_prediction = model(input_example_batch)
  print(example_batch_prediction.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [24]:
print(len(example_batch_prediction))
print(example_batch_prediction)

64
tf.Tensor(
[[[ 1.16113387e-03  6.71223504e-04 -1.53272611e-03 ... -1.20814855e-03
   -3.56934033e-03  2.85322219e-03]
  [-2.30107550e-03  3.62346997e-03 -5.52291749e-03 ...  3.24174849e-04
    8.66473885e-04  3.82824708e-03]
  [ 3.00625805e-03  2.07547680e-03 -6.77848700e-04 ...  1.22490362e-03
   -9.59064346e-05  3.71046667e-03]
  ...
  [ 1.02552613e-02  3.70247383e-03  3.10393446e-03 ... -3.72546259e-03
   -7.75642134e-03 -2.38509523e-03]
  [ 4.43634810e-03  9.57961567e-03 -1.34826847e-03 ...  5.31708170e-03
   -7.40646198e-03 -5.30722504e-03]
  [ 8.70527886e-03  2.22264929e-03 -1.25982740e-03 ... -1.68588827e-03
   -7.30965938e-03 -4.39319480e-03]]

 [[ 1.16113387e-03  6.71223504e-04 -1.53272611e-03 ... -1.20814855e-03
   -3.56934033e-03  2.85322219e-03]
  [ 5.75832557e-03  5.34478435e-03 -9.28938389e-05 ... -3.77976615e-03
   -9.24482476e-04 -1.97509583e-03]
  [ 4.60580410e-03 -6.40970701e-03 -6.41067652e-03 ... -3.29688448e-03
    6.79670973e-03  3.07477359e-03]
  ...
  [ 4.455

In [13]:
# Creating a loss function for 3d array
pred = example_batch_prediction[0]

sampled_indices = tf.random.categorical(pred, num_samples=1)
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars

"!PYCWuiEh;!CBmrFSF:ApgaY3OwSGjnoAd\nwxohYfL!nH\nvbtGT3wZf!LVMEv'YcWK\nSzG&YyUKpp;fvy3EmETRF:VFvTBLp$3V:"

In [14]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [15]:
model.compile(optimizer='adam', loss=loss)

In [23]:
# checkpoint
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
  filepath = checkpoint_prefix,
  save_weights_only = True,
  save_format = 'hdf5'
)

In [17]:
# training
history = model.fit(data, epochs=2, callbacks=[checkpoint_callback])

Epoch 1/2
172/172 [==============================] - 977s 6s/step - loss: 2.5638
Epoch 2/2
172/172 [==============================] - 915s 5s/step - loss: 1.8642


In [18]:
model.save('shakespeare_model.h5')

In [37]:
new_model = tf.keras.models.load_model('shakespare.h5', custom_objects={'loss': loss})

In [38]:
new_model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [39]:
new_model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
new_model.build(tf.TensorShape([1, None]))

In [40]:
checkpoint_num = 2
new_model.load_weights(checkpoint_prefix.format(epoch=checkpoint_num))

In [47]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # low temperatures results in more predictable text.
  # higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 0.6

  # Here batch size == 1
  new_model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)

    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state

    input_eval = tf.expand_dims([predicted_id], 0)
    text_generated.append(idx2char[predicted_id])
  return (start_string + ''.join(text_generated))

In [48]:
inp = input('Type a starting string: ')
print(generate_text(new_model, inp))

romeow.

SICINIUS:
O wive as not that she anger to tee this deeps to my formon where I line a will pay the russ deaven with his promy.

KING RICHARD III:
I prord you so slayer with misten this allace;
And I will dight in the respised and the kings the dears is with hime
And ears more, so dead the vonse, in the have my lord,
And they he wan thou arm furse and strevest in the beed than may it well.
Betsee thou nather and so more in thy shall with here
stall with lively love, it itness.

Sechard:
Non sund so sot, you well me it father pantime for the robe, the good for with your down;
Les sagh the see:
I will the word look--pairs and for here brenger to-mand and man
To will at stay the life so leter throw as like,
And I am know they may the laweral fither or with his like a this
To cranion of his soul not be resides,
In see strow the gentlemant and the head will mather,
I have seep the prochous to say the graces.

ARTOUCHIO:
I am the couls thee it instainst.

LORTHUS:
I will the lany?

DU